In [10]:
using System;
using System.IO;
using System.Text.Json;

public class SpawnSetupName
{
    public required string SetupUUID { get; init; }
    public required string Name { get; init; }
}

var allSpawnSetupNames = JsonSerializer.Deserialize<List<SpawnSetupName>>(File.ReadAllText(Path.Combine(Environment.CurrentDirectory, "spawnsetup_names.txt")));

public class SpawnSetupBodyVariant
{
    public required string SetupUUID { get; init; }
    public required string VariantUUID { get; init; }
}

var allSpawnSetupBodyVariants = JsonSerializer.Deserialize<List<SpawnSetupBodyVariant>>(File.ReadAllText(Path.Combine(Environment.CurrentDirectory, "spawnsetup_bodyvariants.txt")));

public class BodyVariantRootUUID
{
    public required string VariantUUID { get; init; }
    public required string RootUUID { get; init; }
}

var allBodyVariantRootUUIDs = JsonSerializer.Deserialize<List<BodyVariantRootUUID>>(File.ReadAllText(Path.Combine(Environment.CurrentDirectory, "bodyvariant_rootuuids.txt")));

string getSpawnSetupNameForBodyVariant(string bodyVariantUUID)
{
    var firstMapping = allSpawnSetupBodyVariants
        .FirstOrDefault(x => x.VariantUUID == bodyVariantUUID);

    if (firstMapping != null)
    {
        var secondMapping = allSpawnSetupNames
            .FindAll(x => x.SetupUUID == firstMapping.SetupUUID)
            .ToList();

        // Single SpawnSetup UUIDs can map to multiple names. Ensure they all match.
        if (secondMapping.DistinctBy(x => x.Name).Count() == 1)
            return secondMapping.First().Name;
    }

    return "X";
}

// The incoming list may contain duplicate BodyVariants UUIDs belonging to different RootUUIDs. We'll use the RootUUID containing
// the fewest number of UUIDs.
var byHash = new Dictionary<string, (string UUID, string RootUUID, string Name)>();
var bySmallestRootUUIDCount = allBodyVariantRootUUIDs
    .GroupBy(x => x.RootUUID)
    .ToList()
    .OrderBy(x => x.Count());

foreach (var group in bySmallestRootUUIDCount)
{
    foreach (var entry in group)
        byHash.TryAdd(entry.VariantUUID, (entry.VariantUUID, entry.RootUUID, getSpawnSetupNameForBodyVariant(entry.VariantUUID)));
}

// Them dump back to disk
var outputFilePath = Path.Combine(Environment.CurrentDirectory, "combined_body_variant_list.txt");

File.WriteAllLines(outputFilePath, byHash
    .Select(x => x.Value)
    .OrderBy(x => x.UUID)
    .Select(x => $"    [ \"{x.UUID}\", \"{x.RootUUID}\", \"{x.Name}\" ],")
    .Append("]")
    .Prepend("    # [ UUID, Root UUID, Name ]")
    .Prepend("CachedBodyVariants = ["));